In [1]:
!pip install mamkit==0.1.1a2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 81.6 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.0 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch as th
from mamkit.models.audio import TransformerEncoder  # Asegurate que este sea el path correcto
from mamkit.configs.audio import TransformerEncoderConfig
from mamkit.configs.base import ConfigKey
from mamkit.data.datasets import InputMode

# Definir la clave de configuración
config_key = ConfigKey(
    dataset='mmused-fallacy',
    task_name='afc',
    input_mode=InputMode.AUDIO_ONLY,
    tags={'anonymous'}
)

# Cargar la configuración usando el mapeo que ya está definido en BiLSTMMFCCsConfig
config = TransformerEncoderConfig.from_config(key=config_key)

# Crear el modelo
model = TransformerEncoder(
    embedding_dim=config.embedding_dim,
    encoder=config.encoder,  # ✅ este debe ser un callable como `lambda: MyEncoder()`
    head=config.head,        # ✅ también callable
    dropout_rate=config.dropout_rate
)

# Mover a GPU si está disponible
device = th.device("cuda" if th.cuda.is_available() else "cpu")
model = model.to(device)

In [3]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/audioo/train_afc_audio.csv")


print(train_df.head())

                                                Ruta  Etiqueta
0  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         0
1  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         0
2  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         1
3  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         0
4  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         0


In [4]:
# Define el nuevo prefijo
import re
new_prefix = "/kaggle/input/datosss/MMUSED-fallacy"

# Función para limpiar y reemplazar rutas
def fix_path(path_str):
    # Extrae la ruta original desde el WindowsPath(...)
    matches = re.findall(r"WindowsPath\(['\"](.*?)['\"]\)", path_str)
    if matches:
        # Reemplaza la parte de la ruta con el nuevo prefijo
        rel_path = matches[0].split("MMUSED-fallacy")[-1]  # solo lo que viene después de MMUSED-fallacy
        return new_prefix + rel_path.replace("\\", "/")   # Normaliza por si acaso
    return None  # Si no hay match

# Aplica la función a la columna Ruta
train_df["Ruta"] = train_df["Ruta"].apply(fix_path)

# Verifica resultado
print(train_df["Ruta"].head())

0    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
1    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
2    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
3    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
4    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
Name: Ruta, dtype: object


In [5]:
def parse_paths_from_string(path_str):
    """
    Dado un string con la ruta, devuelve una lista con la ruta.
    Si ya es una lista, la devuelve directamente.
    """
    # Si es una cadena, la envolvemos en una lista
    if isinstance(path_str, str):
        return [path_str]
    # Si ya es una lista, la devolvemos tal cual
    elif isinstance(path_str, list):
        return [str(p) for p in path_str]
    else:
        print(f"[ERROR] Tipo no soportado para la ruta: {type(path_str)}")
        return []


In [6]:
from torch.utils.data import Dataset

In [7]:
class AudioDataset(Dataset):
    def __init__(self, dataframe, mfcc_transform):
        self.df = dataframe
        self.transform = mfcc_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path_str = self.df.iloc[idx]['Ruta']
    
        try:
            # Extraer rutas desde el string
            paths = parse_paths_from_string(path_str)
    
            if not paths:
                return None
    
            waveforms = []
            for path in paths:
                waveform, sample_rate = torchaudio.load(path)
                waveforms.append(waveform)
    
            waveform = torch.mean(torch.stack(waveforms), dim=0)
            mfcc = self.transform(waveform)
            mfcc = mfcc.squeeze(0).transpose(0, 1)
    
            label = self.df.iloc[idx]['Etiqueta']
    
            return {
                'inputs': mfcc,
                'label': torch.tensor(label, dtype=torch.long)
            }
    
        except Exception as e:
            print(f"[ERROR] idx {idx}: {e}")
            return None


In [8]:
import torchaudio.transforms as T

mfcc_transform = T.MFCC(
    sample_rate=16000,
    n_mfcc=768,  # ⚠️ poco habitual y probablemente no útil
    melkwargs={'n_fft': 400, 'hop_length': 160, 'n_mels': 768}
)


/usr/local/lib/python3.11/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (768) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [9]:
from torch.utils.data import DataLoader

train_dataset = AudioDataset(train_df, mfcc_transform)


In [10]:
train_dataset[1]

[ERROR] idx 1: name 'torchaudio' is not defined


In [11]:
len(train_dataset)

1228

In [12]:
def collate_fn_pad(batch):
    import torch
    import torch.nn.functional as F


    # Filtrar elementos inválidos y asegurar que sean tensores 2D
    batch = [x for x in batch if x is not None and isinstance(x['inputs'], torch.Tensor) and x['inputs'].ndim == 2]

    if len(batch) == 0:
        print("[SKIP] Batch vacío tras filtrar entradas inválidas.")
        return None

    inputs = []
    labels = []
    lengths = []

    for item in batch:
        x = item['inputs']
        y = item['label']
        inputs.append(x)
        labels.append(y)
        lengths.append(x.shape[0])  # secuencia temporal

    # Padding en la dimensión 0 (timesteps)
    max_len = max(lengths)
    feature_dim = inputs[0].shape[1]

    padded_inputs = [
        F.pad(input, (0, 0, 0, max_len - input.shape[0]))  # pad solo en la dimensión de tiempo
        for input in inputs
    ]

    try:
        inputs_tensor = torch.stack(padded_inputs)  # (batch_size, max_len, feature_dim)
        labels_tensor = torch.tensor(labels)

        # Máscara binaria (1 para datos reales, 0 para padding)
        input_mask = torch.zeros((len(batch), max_len), dtype=torch.float32)
        for i, l in enumerate(lengths):
            input_mask[i, :l] = 1

        return {
            'inputs': inputs_tensor,
            'input_mask': input_mask,
            'label': labels_tensor
        }

    except Exception as e:
        print("Error al hacer el stack:", e)
        return None


In [13]:
from torch.utils.data import random_split

# Supongamos que tienes el dataset original como `train_dataset`
train_size = int(0.8 * len(train_dataset))  # 80% para entrenamiento
val_size = len(train_dataset) - train_size  # 20% para validación

# Dividimos el dataset
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 982
Validation dataset size: 246


In [14]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5221):
        super(PositionalEncoding, self).__init__()

        # Creamos una matriz de posiciones (max_len x d_model)
        self.d_model = d_model
        self.max_len = max_len

        # La codificación posicional se genera con senos y cosenos
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))  # (d_model/2)

        # Aplicamos senos y cosenos en las posiciones
        pe[:, 0::2] = torch.sin(position * div_term)  # las posiciones 0, 2, 4, ...
        pe[:, 1::2] = torch.cos(position * div_term)  # las posiciones 1, 3, 5, ...

        # Añadimos una dimensión adicional para la compatibilidad con el modelo
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x: Tensor de entrada de forma (batch_size, seq_len, d_model)
        """
        seq_len = x.size(1)
        # Aseguramos que la longitud de la secuencia no supere la longitud máxima
        pe = self.pe[:, :seq_len]
        return x + pe


In [15]:
import torch as th
import torchaudio
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm  # Importar tqdm para la barra de progreso
from sklearn.metrics import accuracy_score, f1_score  # Para calcular métricas

# Asegúrate de que el modelo esté en modo de evaluación
# No lo pongas en eval hasta la parte de validación

# Instanciar la codificación posicional
d_model = 768  # Este debe ser el tamaño de las representaciones de entrada del modelo
pos_encoder = PositionalEncoding(d_model=d_model, max_len=5000).to(device)   # Usamos 5000 como un valor máximo para max_len

# Crear un DataLoader para iterar sobre el dataset de entrenamiento y validación
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn_pad)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn_pad)

# Definir un optimizador y una función de pérdida
optimizer = th.optim.Adam(model.parameters(), lr=0.001)  # Optimización
criterion = th.nn.CrossEntropyLoss()  # Función de pérdida para clasificación multiclase

# Medir el tiempo de cada época
for epoch in range(5):
    
    # Inicializar las listas para las predicciones de esta época
    train_predictions = []
    train_true_labels = []
    val_predictions = []
    val_true_labels = []
    
    # ----- Entrenamiento -----
    model.train()
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1} - Training", leave=True)
    for batch in train_loader_tqdm:
        if batch is None:
            continue

        inputs = batch['inputs'].to(device)
        input_mask = batch['input_mask'].to(device)
        labels = batch['label'].to(device)

        # Aplicar codificación posicional a las entradas
        inputs = inputs[:, :5000, :]  # Recortar para que la longitud de la secuencia no supere max_len
        input_mask = input_mask[:, :5000]  # Recortar la máscara para que coincida con max_len
        inputs = pos_encoder(inputs)  # Agregar la codificación posicional

        outputs = model({'inputs': inputs, 'input_mask': input_mask})

        # Obtener la pérdida
        loss = criterion(outputs, labels)

        # Backpropagation y optimización
        optimizer.zero_grad()  # Resetear los gradientes
        loss.backward()  # Calcular los gradientes
        optimizer.step()  # Actualizar los pesos del modelo

        # Obtener las predicciones (puedes aplicar softmax si deseas probabilidades)
        _, preds = th.max(outputs, 1)  # Si la salida del modelo es logits

        # Almacenar las predicciones y las etiquetas reales
        train_predictions.extend(preds.cpu().numpy())
        train_true_labels.extend(labels.cpu().numpy())
        
        # Actualizar la barra de progreso de entrenamiento
        train_loader_tqdm.set_postfix({'train_batch': len(train_predictions)})

    # ----- Validación -----
    model.eval()
    val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch+1} - Validation", leave=True)
    with th.no_grad():  # No calculamos gradientes para la inferencia
        for batch in val_loader_tqdm:
            if batch is None:
                continue
            inputs = batch['inputs'].to(device)
            input_mask = batch['input_mask'].to(device)
            labels = batch['label'].to(device)
    
            # Recortar las secuencias y la máscara para que no superen max_len
            inputs = inputs[:, :5000, :]  # Recortar para que la longitud de la secuencia no supere max_len
            input_mask = input_mask[:, :5000]  # Recortar la máscara para que coincida con max_len
    
            # Aplicar codificación posicional a las entradas
            inputs = pos_encoder(inputs)  # Agregar la codificación posicional
    
            outputs = model({'inputs': inputs, 'input_mask': input_mask})
    
            # Obtener las predicciones (puedes aplicar softmax si deseas probabilidades)
            _, preds = th.max(outputs, 1)  # Si la salida del modelo es logits
    
            # Almacenar las predicciones y las etiquetas reales
            val_predictions.extend(preds.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())
            
            # Actualizar la barra de progreso de validación
            val_loader_tqdm.set_postfix({'val_batch': len(val_predictions)})

    # Calcular y mostrar las métricas después de cada época
    accuracy = accuracy_score(val_true_labels, val_predictions)
    f1 = f1_score(val_true_labels, val_predictions, average='weighted')
    macro_f1 = f1_score(val_true_labels, val_predictions, average='macro')

    print(f"Epoch {epoch+1} completed")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1: {f1:.4f}")
    print(f"Validation Macro F1: {macro_f1:.4f}")


Epoch 1 - Validation: 100%|██████████| 123/123 [00:08<00:00, 14.01it/s, val_batch=246]


Epoch 1 completed
Validation Accuracy: 0.6626
Validation F1: 0.5281
Validation Macro F1: 0.1328


Epoch 2 - Validation: 100%|██████████| 123/123 [00:06<00:00, 18.89it/s, val_batch=246]


Epoch 2 completed
Validation Accuracy: 0.6626
Validation F1: 0.5281
Validation Macro F1: 0.1328


Epoch 3 - Validation: 100%|██████████| 123/123 [00:06<00:00, 19.14it/s, val_batch=246]


Epoch 3 completed
Validation Accuracy: 0.6626
Validation F1: 0.5281
Validation Macro F1: 0.1328


Epoch 4 - Validation: 100%|██████████| 123/123 [00:06<00:00, 19.66it/s, val_batch=246]


Epoch 4 completed
Validation Accuracy: 0.6626
Validation F1: 0.5281
Validation Macro F1: 0.1328


Epoch 5 - Validation: 100%|██████████| 123/123 [00:06<00:00, 18.91it/s, val_batch=246]

Epoch 5 completed
Validation Accuracy: 0.6545
Validation F1: 0.5268
Validation Macro F1: 0.1325


In [16]:
# Medir el tiempo de cada época
for epoch in range(5):
    
    # Inicializar las listas para las predicciones de esta época
    train_predictions = []
    train_true_labels = []
    val_predictions = []
    val_true_labels = []
    
    # ----- Entrenamiento -----
    model.train()
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1} - Training", leave=True)
    for batch in train_loader_tqdm:
        if batch is None:
            continue

        inputs = batch['inputs'].to(device)
        input_mask = batch['input_mask'].to(device)
        labels = batch['label'].to(device)

        # Aplicar codificación posicional a las entradas
        inputs = inputs[:, :5000, :]  # Recortar para que la longitud de la secuencia no supere max_len
        input_mask = input_mask[:, :5000]  # Recortar la máscara para que coincida con max_len
        inputs = pos_encoder(inputs)  # Agregar la codificación posicional

        outputs = model({'inputs': inputs, 'input_mask': input_mask})

        # Obtener la pérdida
        loss = criterion(outputs, labels)

        # Backpropagation y optimización
        optimizer.zero_grad()  # Resetear los gradientes
        loss.backward()  # Calcular los gradientes
        optimizer.step()  # Actualizar los pesos del modelo

        # Obtener las predicciones (puedes aplicar softmax si deseas probabilidades)
        _, preds = th.max(outputs, 1)  # Si la salida del modelo es logits

        # Almacenar las predicciones y las etiquetas reales
        train_predictions.extend(preds.cpu().numpy())
        train_true_labels.extend(labels.cpu().numpy())
        
        # Actualizar la barra de progreso de entrenamiento
        train_loader_tqdm.set_postfix({'train_batch': len(train_predictions)})

    # ----- Validación -----
    model.eval()
    val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch+1} - Validation", leave=True)
    with th.no_grad():  # No calculamos gradientes para la inferencia
        for batch in val_loader_tqdm:
            if batch is None:
                continue
            inputs = batch['inputs'].to(device)
            input_mask = batch['input_mask'].to(device)
            labels = batch['label'].to(device)
    
            # Recortar las secuencias y la máscara para que no superen max_len
            inputs = inputs[:, :5000, :]  # Recortar para que la longitud de la secuencia no supere max_len
            input_mask = input_mask[:, :5000]  # Recortar la máscara para que coincida con max_len
    
            # Aplicar codificación posicional a las entradas
            inputs = pos_encoder(inputs)  # Agregar la codificación posicional
    
            outputs = model({'inputs': inputs, 'input_mask': input_mask})
    
            # Obtener las predicciones (puedes aplicar softmax si deseas probabilidades)
            _, preds = th.max(outputs, 1)  # Si la salida del modelo es logits
    
            # Almacenar las predicciones y las etiquetas reales
            val_predictions.extend(preds.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())
            
            # Actualizar la barra de progreso de validación
            val_loader_tqdm.set_postfix({'val_batch': len(val_predictions)})

    # Calcular y mostrar las métricas después de cada época
    accuracy = accuracy_score(val_true_labels, val_predictions)
    f1 = f1_score(val_true_labels, val_predictions, average='weighted')
    macro_f1 = f1_score(val_true_labels, val_predictions, average='macro')

    print(f"Epoch {epoch+1} completed")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1: {f1:.4f}")
    print(f"Validation Macro F1: {macro_f1:.4f}")


Epoch 1 - Validation: 100%|██████████| 123/123 [00:06<00:00, 18.20it/s, val_batch=246]


Epoch 1 completed
Validation Accuracy: 0.6382
Validation F1: 0.5551
Validation Macro F1: 0.1899


Epoch 2 - Validation: 100%|██████████| 123/123 [00:06<00:00, 19.39it/s, val_batch=246]


Epoch 2 completed
Validation Accuracy: 0.6626
Validation F1: 0.5351
Validation Macro F1: 0.1434


Epoch 3 - Validation: 100%|██████████| 123/123 [00:06<00:00, 19.40it/s, val_batch=246]


Epoch 3 completed
Validation Accuracy: 0.6667
Validation F1: 0.5536
Validation Macro F1: 0.1697


Epoch 4 - Validation: 100%|██████████| 123/123 [00:06<00:00, 19.36it/s, val_batch=246]


Epoch 4 completed
Validation Accuracy: 0.6463
Validation F1: 0.5480
Validation Macro F1: 0.1713


Epoch 5 - Validation: 100%|██████████| 123/123 [00:07<00:00, 17.32it/s, val_batch=246]

Epoch 5 completed
Validation Accuracy: 0.6301
Validation F1: 0.5473
Validation Macro F1: 0.1824


In [17]:
# Imprimir las primeras predicciones
print(val_predictions[:20])
print(val_true_labels[:20])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0]
[0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 3, 0, 2, 0, 0]


In [18]:
class AudioTestDataset(Dataset):
    def __init__(self, dataframe, mfcc_transform):
        self.df = dataframe
        self.transform = mfcc_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path_str = self.df.iloc[idx]['Ruta']
    
        try:
            # Extraer rutas desde el string
            paths = parse_paths_from_string(path_str)
    
            if not paths:
                return None
    
            waveforms = []
            for path in paths:
                waveform, sample_rate = torchaudio.load(path)
                waveforms.append(waveform)
    
            waveform = torch.mean(torch.stack(waveforms), dim=0)
            mfcc = self.transform(waveform)
            mfcc = mfcc.squeeze(0).transpose(0, 1)
    
            return {
                'inputs': mfcc
            }
    
        except Exception as e:
            print(f"[ERROR] idx {idx}: {e}")
            return None


In [19]:
def collate_fn_pad_test(batch):
    import torch
    import torch.nn.functional as F


    # Filtrar elementos inválidos y asegurar que sean tensores 2D
    batch = [x for x in batch if x is not None and isinstance(x['inputs'], torch.Tensor) and x['inputs'].ndim == 2]

    if len(batch) == 0:
        print("[SKIP] Batch vacío tras filtrar entradas inválidas.")
        return None

    inputs = []
    lengths = []

    for item in batch:
        x = item['inputs']
        inputs.append(x)
        lengths.append(x.shape[0])  # secuencia temporal

    # Padding en la dimensión 0 (timesteps)
    max_len = max(lengths)
    feature_dim = inputs[0].shape[1]

    padded_inputs = [
        F.pad(input, (0, 0, 0, max_len - input.shape[0]))  # pad solo en la dimensión de tiempo
        for input in inputs
    ]

    try:
        inputs_tensor = torch.stack(padded_inputs)  # (batch_size, max_len, feature_dim)

        # Máscara binaria (1 para datos reales, 0 para padding)
        input_mask = torch.zeros((len(batch), max_len), dtype=torch.float32)
        for i, l in enumerate(lengths):
            input_mask[i, :l] = 1

        return {
            'inputs': inputs_tensor,
            'input_mask': input_mask,
        }

    except Exception as e:
        print("Error al hacer el stack:", e)
        return None



In [20]:
# Crear el dataset para test (usando la misma transformación)



test_df = pd.read_csv("/kaggle/input/audioo/train_afc_audio.csv")
test_df["Ruta"] = test_df["Ruta"].apply(fix_path)
test_dataset = AudioTestDataset(test_df, mfcc_transform)

# Crear el DataLoader para el conjunto de test
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn_pad_test)

# Ahora puedes realizar la evaluación sobre el conjunto de test

In [21]:
len(test_df)

1228

In [24]:
import numpy as np

model.eval()
test_predictions = []

# Usamos tqdm para mostrar el progreso
test_loader_tqdm = tqdm(test_loader, desc="Test Evaluation", leave=True)

with th.no_grad():
    for batch in test_loader_tqdm:
        if batch is None or 'inputs' not in batch or len(batch['inputs']) == 0:  # Verifica si el batch está vacío o no tiene 'inputs'
            test_predictions.append(0)  # Usamos append para agregar un único valor

        else:
            inputs = batch['inputs'].to(device)
            input_mask = batch['input_mask'].to(device)
    
            # Recortar las secuencias y la máscara para que no superen max_len
            inputs = inputs[:, :5000, :]  # Recortar para que la longitud de la secuencia no supere max_len
            input_mask = input_mask[:, :5000]  # Recortar la máscara para que coincida con max_len

            # Aplicar codificación posicional a las entradas
            inputs = pos_encoder(inputs)  # Agregar la codificación posicional
    
            outputs = model({'inputs': inputs, 'input_mask': input_mask})
            
            _, preds = torch.max(outputs, 1)
            test_predictions.extend(preds.cpu().numpy())  # extend para agregar múltiples predicciones

# Reemplazamos los NaN con 0 (o con la clase mayoritaria que prefieras)
test_predictions = [0 if np.isnan(pred) else pred for pred in test_predictions]

# Ahora `test_predictions` tiene las predicciones con NaN reemplazados por 0


Test Evaluation: 100%|██████████| 1228/1228 [00:30<00:00, 40.67it/s]


In [25]:
# Ahora, puedes guardar las predicciones de test o visualizarlas
print("Test Predictions:", test_predictions)

Test Predictions: [0, 1, 1, 5, 0, 5, 5, 5, 0, 1, 5, 5, 5, 2, 5, 0, 0, 0, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 5, 1, 1, 0, 1, 1, 1, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [26]:
len(test_predictions)

1228

In [27]:
df_predictions = pd.DataFrame(test_predictions, columns=["Prediction"])

# Guardar el DataFrame en un archivo CSV
df_predictions.to_csv('/kaggle/working/BILSTM_audio.csv', index=False)

print("Las predicciones han sido guardadas en 'test_predictions.csv'.")

Las predicciones han sido guardadas en 'test_predictions.csv'.


In [28]:
df_predictions["Prediction"].value_counts()

Prediction
0    1081
2      93
1      35
5      19
Name: count, dtype: int64